<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=6e48b06a5ccbd6bce31d88e4cc7c9144ee8569d7e726128a3a57adb0cafe5ed9
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-10 10:20:57
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.40 C
-------------------
Today PnL: 88.36 K (0.63%)
Current PnL: -28.22 L (-18.23%)
CY Booked + Current PnL: -13.93 L (-9.0%)
-------------------
Total profit:  1.22 L
Total loss:  -29.44 L
-------------------
Total Booked + Current PnL: 13.05 L (10.21%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.2%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 92.85 L (66.29%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 5.13%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,1.16,-17.08,20.71,0.09,17306.0,-17212.0,83565.0,94.02,40.0,M-SC,2.36,253.0,-0.99,0.62,9.79,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,4.32,-11.01,12.39,0.02,19247.0,-19212.0,155347.0,137.17,55.0,M-SC,16.73,234.0,-1.00,1.15,35.13,OX40N,NTT,PAINTS
43,ITC,452.00,0.74,-1.29,11.88,10.44,23470.0,-2582.0,197556.0,-41.15,48.0,X-LC,1.70,5.0,-0.11,1.46,4.20,X40,NTT,FMCG
50,MASFIN,398.61,0.84,-2.13,24.70,22.05,23686.0,-2085.0,95895.0,-15.54,56.0,H-SC,6.87,164.0,-0.09,0.71,39.39,XR,ATH,FINANCE
82,VOLTAS,1530.00,-0.15,4.35,14.70,19.69,29412.0,8343.0,200085.0,-5.78,43.0,X-MC,2.63,78.0,0.28,1.48,11.44,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.0,4.32,-11.01,12.39,0.02,19247.0,-19212.0,155347.0,137.17,55.0,M-SC,16.73,234.0,-1.00,1.15,35.13,OX40N,NTT,PAINTS
44,JCHAC,2282.0,4.07,-35.37,54.71,-0.01,45190.0,-45205.0,82600.0,16813.04,27.0,M-SC,4.12,233.0,-1.00,0.61,5.72,OX40N,NTT,AC
18,COFFEEDAY,80.0,3.75,-44.30,142.87,35.27,90358.0,-50304.0,63245.0,-57.75,31.0,L-SC,10.18,270.0,-0.56,0.47,54.79,XR,NTT,HOTELS
30,HATHWAY,31.4,2.72,-29.03,144.17,73.29,111242.0,-31560.0,77160.0,794.93,44.0,H-SC,7.30,172.0,-0.28,0.57,4.05,XR,NTT,ENTERTAINMENT
49,LAOPALA,464.0,2.66,-38.50,126.16,39.09,101720.0,-50477.0,80628.0,76.43,33.0,H-SC,7.84,155.0,-0.50,0.60,3.84,AR,NTT,CERAMICS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.00,-1.82,-22.13,97.11,53.50,228187.0,-66773.0,234978.0,-62.35,31.0,X-MC,1.24,58.0,-0.29,1.74,4.37,XY24,NTT,FMCG
61,ROUTE,2227.21,-1.12,-48.09,225.69,69.05,155882.0,-63997.0,69069.0,-58.05,48.0,H-SC,23.15,140.0,-0.41,0.51,6.52,SR,ATH,IT
59,RELAXO,1176.00,-0.83,-45.94,181.74,52.31,142613.0,-66689.0,78471.0,-42.41,50.0,X-SC,6.09,91.0,-0.47,0.58,5.04,X40N,NTT,FOOTWEAR
70,SYMPHONY,1306.00,-0.79,-32.38,47.85,-0.03,55372.0,-55422.0,115719.0,-45.70,51.0,M-SC,16.86,206.0,-1.00,0.86,3.03,OX40N,NTT,DURABLES
15,CAMS,4762.00,-0.75,-80.51,540.14,24.79,265193.0,-202759.0,49097.0,-81.32,38.0,X-SC,1.53,86.0,-0.76,0.36,20.02,X40N,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,1.96,-6.19,121.64,107.91,168474.0,-9142.0,138502.0,-23.79,41.0,M-SC,12.15,216.0,-0.05,1.03,3.27,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,4.32,-11.01,12.39,0.02,19247.0,-19212.0,155347.0,137.17,55.0,M-SC,16.73,234.0,-1.00,1.15,35.13,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,1.16,-17.08,20.71,0.09,17306.0,-17212.0,83565.0,94.02,40.0,M-SC,2.36,253.0,-0.99,0.62,9.79,OX40N,NTT,DURABLES
66,SIS,528.00,0.50,-24.28,60.15,21.26,50571.0,-26958.0,84074.0,1983.42,48.0,H-SC,4.69,166.0,-0.53,0.62,13.59,OX40N,NTT,MISC
47,KANSAINER,340.00,1.50,-24.83,50.96,13.47,103299.0,-66960.0,202707.0,-69.09,33.0,H-SC,3.63,158.0,-0.65,1.50,3.07,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.96,6.6,61.61,72.27,111282.0,11176.0,180624.0,-8.35,67.0,M-LC,2.07,99.0,0.1,1.34,13.18,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.96,6.60,61.61,72.27,111282.0,11176.0,180624.0,-8.35,67.0,M-LC,2.07,99.0,0.10,1.34,13.18,XR,NTT,IT
38,INDIAMART,4810.62,0.28,-2.70,112.46,106.72,134955.0,-3333.0,120003.0,-51.77,35.0,H-SC,1.55,139.0,-0.02,0.89,20.04,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,1.70,-1.31,37.43,35.63,77414.0,-2751.0,206823.0,-15.41,46.0,H-MC,3.13,119.0,-0.04,1.53,15.14,AR,ATH,PHARMA
25,FINCABLES,1641.55,1.96,-6.19,121.64,107.91,168474.0,-9142.0,138502.0,-23.79,41.0,M-SC,12.15,216.0,-0.05,1.03,3.27,OX40N,ATH,CABLES
37,IEX,219.00,0.39,-3.81,54.51,48.63,104845.0,-7613.0,192341.0,-36.00,49.0,H-SC,14.06,136.0,-0.07,1.42,7.31,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,0.43,-12.82,39.46,21.58,48210.0,-17964.0,122175.0,-56.03,26.0,X-MC,6.00,56.0,-0.37,0.90,8.16,X40N,ATH,IT
75,TMPV,600.00,0.22,-18.14,73.69,42.18,166738.0,-50140.0,226270.0,-26.74,26.0,X-LC,5.59,3.0,-0.30,1.67,0.22,XY24,NTT,AUTO
9,BAJAJHFL,181.50,1.17,-20.95,87.79,48.45,139152.0,-42000.0,158506.0,-26.53,28.0,X-MC,11.66,64.0,-0.30,1.17,3.20,X40N,ATH,FINANCE
8,AWL,485.00,-1.82,-22.13,97.11,53.50,228187.0,-66773.0,234978.0,-62.35,31.0,X-MC,1.24,58.0,-0.29,1.74,4.37,XY24,NTT,FMCG
33,HINDUNILVR,2922.00,0.26,-10.04,26.35,13.67,60937.0,-25803.0,231260.0,-16.78,31.0,X-LC,2.80,14.0,-0.42,1.71,9.18,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,-1.82,-22.13,97.11,53.50,228187.0,-66773.0,234978.0,-62.35,31.0,X-MC,1.24,58.0,-0.29,1.74,4.37,XY24,NTT,FMCG
15,CAMS,4762.0,-0.75,-80.51,540.14,24.79,265193.0,-202759.0,49097.0,-81.32,38.0,X-SC,1.53,86.0,-0.76,0.36,20.02,X40N,NTT,MISC
43,ITC,452.0,0.74,-1.29,11.88,10.44,23470.0,-2582.0,197556.0,-41.15,48.0,X-LC,1.70,5.0,-0.11,1.46,4.20,X40,NTT,FMCG
3,ACC,3906.0,0.39,-24.69,117.98,64.15,211408.0,-58761.0,179190.0,-55.88,38.0,X-SC,1.96,82.0,-0.28,1.33,0.52,XY24,BTT,CEMENT
36,ICICIPRULI,790.0,1.19,3.38,26.68,30.97,48748.0,5977.0,182715.0,-20.92,55.0,X-MC,2.06,75.0,0.12,1.35,16.37,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,PGHH,17907.65,-0.21,-6.10,42.44,33.76,80033.0,-12240.0,188580.0,-33.72,33.0,X-MC,5.16,57.0,-0.15,1.40,0.00,X40,ATH,FMCG
75,TMPV,600.00,0.22,-18.14,73.69,42.18,166738.0,-50140.0,226270.0,-26.74,26.0,X-LC,5.59,3.0,-0.30,1.67,0.22,XY24,NTT,AUTO
53,PAGEIND,51605.08,0.32,-8.84,38.30,26.07,57167.0,-14480.0,149260.0,-32.90,32.0,X-MC,11.83,55.0,-0.25,1.10,0.32,X40,ATH,APPARELS
3,ACC,3906.00,0.39,-24.69,117.98,64.15,211408.0,-58761.0,179190.0,-55.88,38.0,X-SC,1.96,82.0,-0.28,1.33,0.52,XY24,BTT,CEMENT
31,HAVELLS,2069.16,-0.03,-9.12,45.78,32.48,136458.0,-29917.0,298074.0,-13.77,41.0,X-MC,5.24,67.0,-0.22,2.21,1.26,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.07,-30.01,104.09,42.84,47358.0,-19509.0,45497.0,-54.27,41.0,X-SC,36.82,83.0,-0.41,0.34,2.93,XY24,NTT,MISC
15,CAMS,4762.00,-0.75,-80.51,540.14,24.79,265193.0,-202759.0,49097.0,-81.32,38.0,X-SC,1.53,86.0,-0.76,0.36,20.02,X40N,NTT,MISC
59,RELAXO,1176.00,-0.83,-45.94,181.74,52.31,142613.0,-66689.0,78471.0,-42.41,50.0,X-SC,6.09,91.0,-0.47,0.58,5.04,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.40,-36.26,112.11,35.20,91948.0,-46654.0,82016.0,7.58,41.0,X-SC,15.39,92.0,-0.51,0.61,4.18,X40,NTT,FOOTWEAR
22,DIXON,18931.72,0.43,-12.82,39.46,21.58,48210.0,-17964.0,122175.0,-56.03,26.0,X-MC,6.00,56.0,-0.37,0.90,8.16,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.08,-12.90,36.94,19.28,111314.0,-44620.0,301336.0,-23.95,63.0,X-LC,5.48,1.0,-0.40,2.23,11.40,X40,ATH,IT
41,INFY,2275.00,-0.13,8.71,42.46,54.87,144423.0,27252.0,340140.0,-15.29,63.0,X-LC,3.30,2.0,0.19,2.52,17.71,X40,BTT,IT
75,TMPV,600.00,0.22,-18.14,73.69,42.18,166738.0,-50140.0,226270.0,-26.74,26.0,X-LC,5.59,3.0,-0.30,1.67,0.22,XY24,NTT,AUTO
81,VBL,671.64,1.23,-3.58,40.70,35.67,123951.0,-11293.0,304549.0,-14.99,56.0,X-LC,4.71,4.0,-0.09,2.25,9.78,X40N,ATH,FMCG
43,ITC,452.00,0.74,-1.29,11.88,10.44,23470.0,-2582.0,197556.0,-41.15,48.0,X-LC,1.70,5.0,-0.11,1.46,4.20,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.20,-37.54,112.77,32.89,54380.0,-28984.0,48222.0,-695.74,55.0,L-MC,5.74,259.0,-0.53,0.36,32.80,XR,NTT,BANKS
6,ASIANTILES,137.00,2.32,-17.68,120.68,81.67,93199.0,-16582.0,77228.0,6938.89,40.0,L-SC,14.30,271.0,-0.18,0.57,49.55,XR,NTT,CERAMICS
50,MASFIN,398.61,0.84,-2.13,24.70,22.05,23686.0,-2085.0,95895.0,-15.54,56.0,H-SC,6.87,164.0,-0.09,0.71,39.39,XR,ATH,FINANCE
13,BSOFT,831.70,-0.19,-22.11,93.60,50.79,102537.0,-31105.0,109548.0,-0.27,69.0,H-SC,5.86,151.0,-0.30,0.81,27.55,XR,ATH,IT
39,INDIGOPNTS,1408.00,4.32,-11.01,12.39,0.02,19247.0,-19212.0,155347.0,137.17,55.0,M-SC,16.73,234.0,-1.00,1.15,35.13,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-0.19,-22.11,93.60,50.79,102537.0,-31105.0,109548.0,-0.27,69.0,H-SC,5.86,151.0,-0.3,0.81,27.55,XR,ATH,IT
39,INDIGOPNTS,1408.00,4.32,-11.01,12.39,0.02,19247.0,-19212.0,155347.0,137.17,55.0,M-SC,16.73,234.0,-1.0,1.15,35.13,OX40N,NTT,PAINTS
32,HCLTECH,1922.01,0.66,8.28,15.19,24.73,39791.0,20030.0,261954.0,13.18,64.0,X-LC,5.98,7.0,0.5,1.94,24.78,X40,ATH,IT
84,WIPRO,420.00,0.96,6.60,61.61,72.27,111282.0,11176.0,180624.0,-8.35,67.0,M-LC,2.07,99.0,0.1,1.34,13.18,XR,NTT,IT
56,RAJESHEXPO,518.00,1.31,-63.12,171.36,0.07,86684.0,-86591.0,50586.0,1753.54,56.0,L-SC,2.32,268.0,-1.0,0.37,25.81,OX40N,NTT,JEWELLERY


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.74
1,25,45.68
2,50,76.58


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.18
MC    30.24
LC    24.58
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.30
X40      22.43
X40N     10.89
XR       10.09
XY25      9.35
AR        9.33
OX40N     7.83
SR        1.03
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.55
X-MC    23.50
X-LC    21.04
M-SC    12.20
X-SC     7.02
H-MC     4.97
M-MC     1.41
L-SC     1.41
M-LC     1.34
H-LC     1.16
L-LC     1.04
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.38,-7.09,41.73
IT,12.80,-16.55,79.56
FINANCE,9.53,-20.40,71.28
ELECTRICAL,6.21,-11.79,52.74
PAINTS,6.15,-11.22,27.74
MISC,6.10,-57.65,122.23
INSURANCE,4.59,-1.98,37.92
PHARMA,4.11,-0.55,32.74
AUTO,3.40,-24.89,78.44


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3214520.0,21
AR,1311910.0,10
XR,1299920.0,13
X40,1008794.0,14
X40N,974523.0,9
OX40N,735291.0,10
XY25,376376.0,6
SR,280612.0,2
MH,83125.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3663111.0,25
M-SC,1412917.0,15
X-MC,1305747.0,16
X-SC,972439.0,8
X-LC,917932.0,11
H-MC,406805.0,3
L-SC,270241.0,3
M-LC,111282.0,1
H-LC,76750.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1283731.0      6
           AR         895959.0      5
M-SC       XY24       805826.0      6
H-SC       XR         776666.0      7
X-SC       X40N       503081.0      3
X-MC       X40        456661.0      7
           XY24       391173.0      3
X-LC       X40        375067.0      5
H-SC       OX40N      343018.0      4
M-SC       OX40N      305589.0      5
X-SC       XY24       292292.0      3
H-SC       SR         280612.0      2
X-MC       X40N       272455.0      4
X-LC       XY24       246427.0      2
H-MC       AR         211734.0      2
X-LC       X40N       198987.0      2
H-MC       XY24       195071.0      1
X-MC       XY25       185458.0      2
L-SC       XR         183557.0      2
X-SC       X40        177066.0      2
M-SC       XR         174035.0      2
           AR         127467.0      2
M-LC       XR         111282.0      1
X-LC       XY25        97451.0      2
L-SC       OX40N       86684.0      1
H-SC       MH          83125.0      1
H-LC       AR          76750.0      1
M-MC       XY25        54997.0      1
L-MC       XR          54380.0      1
L-LC       XY25        38470.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
